# Main

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import random_split
from torch.utils.data import DataLoader

import lego_classifier.dataset as tfs

from torchvision import datasets, transforms
from torchvision import models

import json

# Load dataset
base_ds = datasets.ImageFolder('LEGO_brick_images_v1', transform=None)

classes = base_ds.classes

# 2. Build a class→index dict (ImageFolder already does this, but we re-create it)
class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

# 3. Persist it
with open('class_to_idx.json', 'w') as f:
    json.dump(class_to_idx, f, indent=2)

train_size = int(0.8 * len(base_ds))
val_size   = len(base_ds) - train_size
train_idx, val_idx = random_split(range(len(base_ds)),
                                  [train_size, val_size],
                                  generator=torch.Generator().manual_seed(42))

train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_ds = tfs.SubsetWithTransform(base_ds, train_idx, train_tf)
val_ds   = tfs.SubsetWithTransform(base_ds, val_idx, val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4)

ModuleNotFoundError: No module named 'lego_classifier'

In [ ]:
from torchvision.models import ResNet18_Weights

# Load pretrained model
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_classes = len(base_ds.classes)  # based on subfolder count
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)  # only train last layer

In [32]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = total_correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss    += loss.item() * images.size(0)
        total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    avg_acc  = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

def evaluate(model, loader):
    model.eval()
    total_loss = total_correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss    += loss.item() * images.size(0)
            total_correct += (outputs.argmax(1) == labels).sum().item()

    avg_loss = total_loss / len(loader.dataset)
    avg_acc  = total_correct / len(loader.dataset)
    return avg_loss, avg_acc

In [35]:
best_val_loss = float('inf')
num_epochs = 10

for epoch in range(1, num_epochs+1):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_loss,   val_acc   = evaluate(model,   val_loader)
    print(f"Epoch {epoch:2d} | "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val   loss {val_loss:.4f}, acc {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_resnet18.pth")

Epoch  1 | Train loss 1.3947, acc 0.5893 | Val   loss 0.7554, acc 0.7939
Epoch  2 | Train loss 0.7191, acc 0.7874 | Val   loss 0.5629, acc 0.8378
Epoch  3 | Train loss 0.5657, acc 0.8230 | Val   loss 0.4565, acc 0.8683
Epoch  4 | Train loss 0.4948, acc 0.8448 | Val   loss 0.4349, acc 0.8644
Epoch  5 | Train loss 0.4415, acc 0.8644 | Val   loss 0.4208, acc 0.8652
Epoch  6 | Train loss 0.4133, acc 0.8677 | Val   loss 0.3686, acc 0.8809
Epoch  7 | Train loss 0.3768, acc 0.8763 | Val   loss 0.3399, acc 0.8785
Epoch  8 | Train loss 0.3561, acc 0.8832 | Val   loss 0.3376, acc 0.9005
Epoch  9 | Train loss 0.3433, acc 0.8811 | Val   loss 0.3246, acc 0.8856
Epoch 10 | Train loss 0.3260, acc 0.8930 | Val   loss 0.3136, acc 0.8864


# Inference example

In [ ]:
with open("class_to_idx.json") as f:
    class_to_idx = json.load(f)

# 1. Load your model weights
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes=len(class_to_idx)) 
model.load_state_dict(torch.load('best_resnet18.pth'))
model.eval()

# 2. Reload the class↔idx map
with open('class_to_idx.json') as f:
    class_to_idx = json.load(f)
# build the inverse map
idx_to_class = {v: k for k, v in class_to_idx.items()}

SyntaxError: invalid syntax (1477799069.py, line 3)